In [1]:
import kagglehub
from docutils.nodes import Sequential
from keras import Input
from polars.dependencies import pandas
from sklearn.model_selection import train_test_split

path = kagglehub.dataset_download("seddiktrk/cafa6-protein-embeddings-esm2")
print("Path to dataset files:", path)

100%|██████████| 1.27G/1.27G [06:06<00:00, 3.72MB/s]

Extracting files...


Path to dataset files: /home/gabriel/.cache/kagglehub/datasets/seddiktrk/cafa6-protein-embeddings-esm2/versions/1


In [3]:
import numpy as np
import pandas as pd
from pathlib import Path

data_dir = Path('/home/gabriel/.cache/kagglehub/datasets/seddiktrk/cafa6-protein-embeddings-esm2/versions/1')

protein_ids = pd.read_csv(data_dir / "protein_ids.csv")["protein_id"].tolist()
embeddings = np.load(data_dir / "protein_embeddings.npy")


print(f"Loaded {len(protein_ids)} embeddings of dimension {embeddings.shape[1]}")

pid_emb_dict = {}
for pid, emb in zip(protein_ids, embeddings):
    pid_emb_dict[str(pid)] = emb

Loaded 287001 embeddings of dimension 1280


In [12]:
protein_ids[:10]

['Q9ZSA8',
 'P25353',
 'A0A2R8YCW8',
 'G3V5N8',
 'A0A140LFN4',
 'B8ZZU6',
 'Q01850',
 'P11076',
 'Q9VJ64',
 'Q7YSJ4']

In [21]:
len(protein_ids)

287001

In [16]:
embeddings.shape

(287001, 1280)

In [14]:
pid_emb_dict

{'Q9ZSA8': array([-0.09229109, -0.06628396, -0.01226195, ..., -0.16065988,
         0.0159197 ,  0.00173827], shape=(1280,), dtype=float32),
 'P25353': array([ 0.01162435, -0.03031761, -0.00580197, ..., -0.15341142,
         0.00663222,  0.00142484], shape=(1280,), dtype=float32),
 'A0A2R8YCW8': array([ 0.02737274, -0.04104703, -0.02920537, ..., -0.05123704,
        -0.01053242,  0.02256369], shape=(1280,), dtype=float32),
 'G3V5N8': array([ 0.03376611, -0.07888931, -0.05974137, ..., -0.10354079,
         0.01092495,  0.1584017 ], shape=(1280,), dtype=float32),
 'A0A140LFN4': array([ 0.0119482 , -0.00210759, -0.08492219, ..., -0.05035866,
        -0.05547693,  0.02029048], shape=(1280,), dtype=float32),
 'B8ZZU6': array([ 0.02013669, -0.13927373, -0.04045331, ..., -0.2279436 ,
         0.14340197,  0.065526  ], shape=(1280,), dtype=float32),
 'Q01850': array([ 0.02601305, -0.08974387, -0.02024045, ..., -0.08277065,
         0.14049718,  0.07697625], shape=(1280,), dtype=float32),
 'P11

In [44]:
df = pd.DataFrame(pid_emb_dict).T

In [45]:
df.shape

(287001, 1280)

In [46]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279
Q9ZSA8,-0.092291,-0.066284,-0.012262,0.044118,-0.096111,-0.028308,-0.074759,0.100977,0.024241,0.083448,...,0.003669,-0.076738,-0.023574,0.013527,-0.104102,-0.047541,0.043210,-0.160660,0.015920,0.001738
P25353,0.011624,-0.030318,-0.005802,0.029026,0.028326,-0.034560,0.003770,-0.081593,-0.050148,-0.007592,...,0.082000,0.007247,-0.062506,0.095455,-0.063068,-0.010131,0.042478,-0.153411,0.006632,0.001425
A0A2R8YCW8,0.027373,-0.041047,-0.029205,0.028087,-0.012250,-0.066159,0.030800,0.039873,0.000206,0.055686,...,0.083869,0.008130,-0.122658,-0.020470,0.016220,-0.014810,0.069728,-0.051237,-0.010532,0.022564
G3V5N8,0.033766,-0.078889,-0.059741,0.045629,-0.060334,-0.015612,0.059876,-0.038531,-0.001667,0.075625,...,0.029910,-0.010136,-0.007829,0.052135,0.009843,-0.006972,0.037077,-0.103541,0.010925,0.158402
A0A140LFN4,0.011948,-0.002108,-0.084922,0.068762,0.002155,-0.093367,0.114656,0.040136,0.067143,0.073344,...,-0.030091,0.021493,-0.246086,0.030696,-0.050064,0.078881,0.008374,-0.050359,-0.055477,0.020290


In [48]:
df[0].head()

Q9ZSA8       -0.092291
P25353        0.011624
A0A2R8YCW8    0.027373
G3V5N8        0.033766
A0A140LFN4    0.011948
Name: 0, dtype: float32

In [49]:
df.columns

RangeIndex(start=0, stop=1280, step=1)

In [60]:
df.index.duplicated().sum()

np.int64(0)

In [89]:
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.utils import to_categorical

In [54]:
x_train, x_test, y_train, y_test = train_test_split(df, df.index.to_numpy(), test_size=0.2, random_state=42)

In [90]:
y_train = to_categorical(y_train, num_classes=1280)
y_test = to_categorical(y_test, num_classes=1280)

ValueError: invalid literal for int() with base 10: 'Q8VDK4'

In [56]:
x_train.shape

(229600, 1280)

In [55]:
x_test.shape

(57401, 1280)

In [57]:
y_train.shape

(229600,)

In [58]:
y_test.shape

(57401,)

In [86]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [84]:
model = Sequential([
    Input((1280,)),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(1280, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [87]:
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(32,), output.shape=(32, 1280)